In [8]:
import math
import random
from typing import Union, Tuple, Callable, Set, List

In [9]:
class Value:
    """
    Class to represent a value in a computational graph, supporting operations like addition,
    multiplication, exponentiation, division, negation, and the hyperbolic tangent function.
    It also supports automatic differentiation via the backward method.
    """

    def __init__(self, data: float, _children: Tuple['Value', ...] = (), _op: str = '', label: str = ''):
        """
        Initialize a Value object.

        :param data: The numerical data associated with this value.
        :param _children: The parent nodes contributing to this value in the computational graph.
        :param _op: The operation that produced this value.
        :param label: An optional label for this value.
        """
        self.data = data
        self.grad = 0.0
        self._backward: Callable[[], None] = lambda: None
        self._prev: Set['Value'] = set(_children)
        self._op = _op
        self.label = label


    def __repr__(self) -> str:
        """
        Return a string representation of the Value object.

        :return: A string representation of the Value object.
        """
        return f"Value(data={self.data})"


    def __add__(self, other: Union['Value', float]) -> 'Value':
        """
        Add another value or float to this value.

        :param other: Another value or a float to add.
        :return: The result of the addition as a new Value object.
        """
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward

        return out
    

    def __radd__(self, other: Union['Value', float]) -> 'Value':
        """
        Add another value or float to this value (reverse addition).

        :param other: Another value or a float to add.
        :return: The result of the addition as a new Value object.
        """
        return self + other


    def __mul__(self, other: Union['Value', float]) -> 'Value':
        """
        Multiply this value by another value or float.

        :param other: Another value or a float to multiply by.
        :return: The result of the multiplication as a new Value object.
        """
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward

        return out


    def __rmul__(self, other: Union['Value', float]) -> 'Value':
        """
        Multiply this value by another value or float (reverse multiplication).

        :param other: Another value or a float to multiply by.
        :return: The result of the multiplication as a new Value object.
        """
        return self * other


    def __pow__(self, other: Union[int, float]) -> 'Value':
        """
        Raise this value to the power of another value or float.

        :param other: The exponent, which must be an int or float.
        :return: The result of the exponentiation as a new Value object.
        """
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data ** other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad

        out._backward = _backward

        return out


    def __truediv__(self, other: Union['Value', float]) -> 'Value':
        """
        Divide this value by another value or float.

        :param other: Another value or a float to divide by.
        :return: The result of the division as a new Value object.
        """
        return self * other**-1


    def __neg__(self) -> 'Value':
        """
        Negate this value.

        :return: The negated value as a new Value object.
        """
        return self * -1


    def __sub__(self, other: Union['Value', float]) -> 'Value':
        """
        Subtract another value or float from this value.

        :param other: Another value or a float to subtract.
        :return: The result of the subtraction as a new Value object.
        """
        return self + (-other)


    def tanh(self) -> 'Value':
        """
        Compute the hyperbolic tangent of this value.

        :return: The result of the tanh function as a new Value object.
        """
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t ** 2) * out.grad

        out._backward = _backward

        return out



    def exp(self) -> 'Value':
        """
        Compute the exponential of this value.

        :return: The result of the exp function as a new Value object.
        """
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad

        out._backward = _backward

        return out


    def backward(self) -> None:
        """
        Perform backpropagation to compute the gradients of this value with respect to all
        preceding values in the computational graph.
        """
        topo = []
        visited = set()

        def build_topo(v: 'Value') -> None:
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [11]:

class Neuron:
    """
    A single neuron in a neural network, with a set of weights and a bias.
    """

    def __init__(self, nin: int):
        """
        Initialize a Neuron with random weights and a bias.

        :param nin: Number of input connections to the neuron.
        """
        self.w: List[Value] = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b: Value = Value(random.uniform(-1, 1))

    def __call__(self, x: List[Value]) -> Value:
        """
        Compute the output of the neuron given an input.

        :param x: List of input values.
        :return: The output value after applying the tanh activation function.
        """
        # Weighted sum of inputs plus bias
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self) -> List[Value]:
        """
        Get all parameters of the neuron (weights and bias).

        :return: A list containing the weights and bias.
        """
        return self.w + [self.b]

class Layer:
    """
    A layer in a neural network, consisting of multiple neurons.
    """

    def __init__(self, nin: int, nout: int):
        """
        Initialize a Layer with a specified number of input and output connections.

        :param nin: Number of input connections to each neuron.
        :param nout: Number of neurons in this layer.
        """
        self.neurons: List[Neuron] = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x: List[Value]) -> Union[Value, List[Value]]:
        """
        Compute the output of the layer given an input.

        :param x: List of input values.
        :return: The output values from the layer.
        """
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self) -> List[Value]:
        """
        Get all parameters of the layer.

        :return: A list containing all the parameters (weights and biases) of the neurons in the layer.
        """
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    """
    A multi-layer perceptron (MLP) neural network, consisting of multiple layers.
    """

    def __init__(self, nin: int, nouts: List[int]):
        """
        Initialize an MLP with a specified number of input connections and a list of output connections per layer.

        :param nin: Number of input connections to the MLP.
        :param nouts: List specifying the number of output connections for each layer.
        """
        sz = [nin] + nouts
        self.layers: List[Layer] = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]

    def __call__(self, x: List[Value]) -> Union[Value, List[Value]]:
        """
        Compute the output of the MLP given an input.

        :param x: List of input values.
        :return: The output values from the MLP.
        """
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self) -> List[Value]:
        """
        Get all parameters of the MLP.

        :return: A list containing all the parameters (weights and biases) of the neurons in all layers of the MLP.
        """
        return [p for layer in self.layers for p in layer.parameters()]


In [12]:
x = [2.0, 3.0, -1.0]
mlp = MLP(5, [4, 4, 1])
mlp(x)

Value(data=0.9222548123958281)

In [60]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets
for k in range(20):
  
  # forward pass
  ypred = [mlp(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  for p in mlp.parameters():
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in mlp.parameters():
    p.data += -0.1 * p.grad
  
  print(k, loss.data)


0 0.0003605320228457257
1 0.0003601014865030883
2 0.0003596719619233537
3 0.00035924344556235885
4 0.0003588159338924078
5 0.00035838942340222824
6 0.000357963910596814
7 0.0003575393919973982
8 0.00035711586414130254
9 0.00035669332358187434
10 0.0003562717668883871
11 0.0003558511906459517
12 0.00035543159145541613
13 0.0003550129659332925
14 0.00035459531071165224
15 0.0003541786224380243
16 0.0003537628977753493
17 0.0003533481334018411
18 0.0003529343260109461
19 0.00035252147231122123


In [62]:
ypred

[Value(data=0.9922222339263994),
 Value(data=-0.9921710170481529),
 Value(data=-0.9896168639431333),
 Value(data=0.9889128299922906)]